In [98]:
import numpy as np 
import pandas as pd 
import os 

In [99]:
os.listdir("data")

['106_2020-01-01T0000_2024-05-17T0000.csv',
 '1_2020-01-01T0000_2024-05-17T0000.csv',
 '244_2020-01-01T0000_2024-05-17T0000.csv',
 '262_2020-01-01T0000_2024-05-17T0000.csv',
 '287_2020-01-01T0000_2024-05-17T0000.csv',
 '288_2020-01-01T0000_2024-05-17T0000.csv',
 '289_2020-01-01T0000_2024-05-17T0000.csv',
 '290_2020-01-01T0000_2024-05-17T0000.csv',
 '320_2020-01-01T0000_2024-05-17T0000.csv',
 '327_2020-01-01T0000_2024-05-17T0000.csv',
 '329_2020-01-01T0000_2024-05-17T0000.csv',
 'data_preparing.ipynb',
 'FI',
 'idf',
 'Oomi Spot-hintatieto (1_06_2023-30_09_2023).csv',
 'Oomi Spot-hintatieto (1_10_2023-31_10_23).csv',
 'Oomi Spot-hintatieto (1_11_2023-31_12_2023).csv',
 'Oomi Spot-hintatieto (1_1_2023-31_05_2023).csv',
 'Oomi Spot-hintatieto (1_1_2024-31_03_2024).csv',
 'Oomi Spot-hintatieto (2021).csv',
 'Oomi Spot-hintatieto (2022).csv']

In [100]:
crossborder = pd.read_csv("data/287_2020-01-01T0000_2024-05-17T0000.csv")
regup = pd.read_csv("data/244_2020-01-01T0000_2024-05-17T0000.csv")
regdwn = pd.read_csv("data/106_2020-01-01T0000_2024-05-17T0000.csv")
rrs = pd.read_csv("data/288_2020-01-01T0000_2024-05-17T0000.csv")
nspin = pd.read_csv("data/1_2020-01-01T0000_2024-05-17T0000.csv")


In [101]:
nspin

,Start time UTC,End time UTC,"Automatic Frequency Restoration Reserve, capacity, up"
0,2019-12-31T23:00:00.000Z,2020-01-01T00:00:00.000Z,0
1,2020-01-01T00:00:00.000Z,2020-01-01T01:00:00.000Z,0
2,2020-01-01T01:00:00.000Z,2020-01-01T02:00:00.000Z,0
3,2020-01-01T02:00:00.000Z,2020-01-01T03:00:00.000Z,0
4,2020-01-01T03:00:00.000Z,2020-01-01T04:00:00.000Z,0
...,...,...,...
37918,2024-05-16T18:00:00.000Z,2024-05-16T19:00:00.000Z,25
37919,2024-05-16T19:00:00.000Z,2024-05-16T20:00:00.000Z,31
37920,2024-05-16T20:00:00.000Z,2024-05-16T21:00:00.000Z,52
37921,2024-05-16T21:00:00.000Z,2024-05-16T22:00:00.000Z,68


In [104]:
def transform_df(df, name):
    value_col = df.columns[-1]
    df = df.drop("Start time UTC", axis=1)
    df["End time UTC"] = pd.to_datetime(df["End time UTC"])
    # extract Delivery date and hour
    df["Delivery Date"] = df["End time UTC"].dt.date
    df["Hour Ending"] = df["End time UTC"].dt.time.apply(lambda x: x.strftime("%H:%M"))
    df = df.drop("End time UTC", axis=1)
    df[f"{name}"] = df[value_col]
    df = df.drop(value_col, axis=1)
    df["Repeated Hour Flag"] = df.groupby(["Delivery Date", "Hour Ending"])[name].transform("count").apply(lambda x: "Y" if x > 1 else "N")
    
    return df

In [105]:
nspin = transform_df(nspin, "NSPIN")
regup = transform_df(regup, "REGUP")
regdwn = transform_df(regdwn, "REGDN")
rrs = transform_df(rrs, "RRS")
crossborder = transform_df(crossborder, "CROSSBORDER")

In [106]:
[x.shape for x in [nspin, regup, regdwn, rrs, crossborder]]

[(37923, 4), (38351, 4), (38351, 4), (38352, 4), (29567, 4)]

In [107]:
[(x["Delivery Date"].min(), x["Delivery Date"].max()) for x in [nspin, regup, regdwn, rrs, crossborder]]

[(datetime.date(2020, 1, 1), datetime.date(2024, 5, 16)),
 (datetime.date(2020, 1, 1), datetime.date(2024, 5, 16)),
 (datetime.date(2020, 1, 1), datetime.date(2024, 5, 16)),
 (datetime.date(2020, 1, 1), datetime.date(2024, 5, 16)),
 (datetime.date(2021, 1, 1), datetime.date(2024, 5, 16))]

In [108]:
rrs

,Delivery Date,Hour Ending,RRS,Repeated Hour Flag
0,2020-01-01,00:00,52.2,N
1,2020-01-01,01:00,52.2,N
2,2020-01-01,02:00,52.2,N
3,2020-01-01,03:00,52.2,N
4,2020-01-01,04:00,59.1,N
...,...,...,...,...
38347,2024-05-16,19:00,20.9,N
38348,2024-05-16,20:00,22.1,N
38349,2024-05-16,21:00,22.1,N
38350,2024-05-16,22:00,27.7,N


In [109]:
# concatenate all dataframes on Delivery Date and Hour Ending
df = nspin.merge(regup, on=["Delivery Date", "Hour Ending", "Repeated Hour Flag"])
df = df.merge(regdwn, on=["Delivery Date", "Hour Ending", "Repeated Hour Flag"])
df = df.merge(rrs, on=["Delivery Date", "Hour Ending", "Repeated Hour Flag"])
df = df.merge(crossborder, on=["Delivery Date", "Hour Ending", "Repeated Hour Flag"])


In [110]:
df.columns

Index(['Delivery Date', 'Hour Ending', 'NSPIN', 'Repeated Hour Flag', 'REGUP',
       'REGDN', 'RRS', 'CROSSBORDER'],
      dtype='object')

In [111]:
df = df[['Delivery Date', 'Hour Ending', 'Repeated Hour Flag', 'REGDN','REGUP', 'RRS', 'NSPIN', 'CROSSBORDER']]

In [112]:
df

,Delivery Date,Hour Ending,Repeated Hour Flag,REGDN,REGUP,RRS,NSPIN,CROSSBORDER
0,2021-01-01,01:00,N,24.35000,24.35000,84.09999,0,0.0
1,2021-01-01,02:00,N,23.98000,23.98000,84.09999,0,0.0
2,2021-01-01,03:00,N,23.71999,23.71999,84.09999,0,0.0
3,2021-01-01,04:00,N,23.73000,23.73000,85.79999,0,0.0
4,2021-01-01,05:00,N,24.05999,24.05999,85.90000,60,0.0
...,...,...,...,...,...,...,...,...
29133,2024-05-16,19:00,N,91.89000,430.00000,20.90000,25,40.3
29134,2024-05-16,20:00,N,114.48000,114.48000,22.10000,31,78.2
29135,2024-05-16,21:00,N,84.27000,84.27000,22.10000,52,38.7
29136,2024-05-16,22:00,N,4.30000,4.30000,27.70000,68,39.7


In [113]:
df.duplicated().sum()

0

In [114]:
df["Repeated Hour Flag"].value_counts()

Repeated Hour Flag
N    29138
Name: count, dtype: int64

In [115]:
def get_data_by_year_and_month(df, year, month_name):
    return df.loc[((pd.to_datetime(df["Delivery Date"]).dt.year==year) & (pd.to_datetime(df["Delivery Date"]).dt.month_name().apply(lambda x: x[:3])==month_name))]

In [122]:
get_data_by_year_and_month(df, 2023, "Feb").shape

(672, 8)

In [123]:
get_data_by_year_and_month(df, 2023, "Apr").groupby("Delivery Date")["Hour Ending"].count().unique()

array([24], dtype=int64)

In [118]:
for year in range(2021, 2025, 1):
    sub_df = df[pd.to_datetime(df["Delivery Date"]).dt.year == year]    
    sub_df.to_csv(f"data/FI/CCP/{year}/rpt.00013091.0000000000000000.20210101080006.DAMASMCPC_{year}.csv", index=False)